In [ ]:
import sys
import copy

N, M = map(int, sys.stdin.readline().strip().split())
office = [list(map(int, sys.stdin.readline().strip().split())) for _ in range(N)]

# 사각지대 count 함수
def check_blind(office):
    blind = 0
    for row in office:
        for cell in row:
            if cell == 0:
                blind += 1
    return blind

# CCTV 감시 영역 표시 함수
def trans_office(office, pos, dir):
    x, y = pos
    for dx, dy in dir:
        nx, ny = x + dx, y + dy
        while 0 <= nx < len(office) and 0 <= ny < len(office[0]):
            if office[nx][ny] == 6:  # 벽이면 종료
                break
            if office[nx][ny] == 0:
                office[nx][ny] = '#'  # 감시된 영역 표시
            nx += dx
            ny += dy
    return office

# 최소 사각지대 초기화
min_blind = float('inf')

# 전체 사무실을 순회
for x in range(len(office)):
    for y in range(len(office[0])):
        cctv = office[x][y]

        if cctv == 0 or cctv == 6:
            continue

        # 방향 설정
        if cctv == 1:
            dirs = [[(1, 0)], [(-1, 0)], [(0, 1)], [(0, -1)]]
        elif cctv == 2:
            dirs = [[(1, 0), (-1, 0)], [(0, 1), (0, -1)]]
        elif cctv == 3:
            dirs = [[(0, 1), (1, 0)], [(1, 0), (0, -1)], [(0, -1), (-1, 0)], [(-1, 0), (0, 1)]]
        elif cctv == 4:
            dirs = [[(-1, 0), (0, 1), (1, 0)], [(0, 1), (1, 0), (0, -1)],
                    [(1, 0), (0, -1), (-1, 0)], [(0, -1), (-1, 0), (0, 1)]]
        else:  # cctv == 5
            dirs = [[(-1, 0), (0, 1), (1, 0), (0, -1)]]

        best_blind = float('inf')
        best_office = None

        for d in dirs:
            copied = copy.deepcopy(office)
            new_office = trans_office(copied, (x, y), d)
            blind = check_blind(new_office)

            if blind < best_blind:
                best_blind = blind
                best_office = new_office

        office = best_office
        if best_blind < min_blind:
            min_blind = best_blind

print(min_blind)


In [ ]:
import sys
import copy

# 입력 파싱
def parse_input():
    N, M = map(int, sys.stdin.readline().strip().split())
    office = [list(map(int, sys.stdin.readline().strip().split())) for _ in range(N)]
    cctvs = [(x, y, office[x][y])
             for x in range(N)
             for y in range(M)
             if 1 <= office[x][y] <= 5]
    return N, M, office, cctvs

# CCTV 방향 사전
def get_cctv_directions():
    return {
        1: [[(1, 0)], [(-1, 0)], [(0, 1)], [(0, -1)]],
        2: [[(1, 0), (-1, 0)], [(0, 1), (0, -1)]],
        3: [[(0, 1), (1, 0)], [(1, 0), (0, -1)], [(0, -1), (-1, 0)], [(-1, 0), (0, 1)]],
        4: [[(-1, 0), (0, 1), (1, 0)], [(0, 1), (1, 0), (0, -1)],
            [(1, 0), (0, -1), (-1, 0)], [(0, -1), (-1, 0), (0, 1)]],
        5: [[(-1, 0), (0, 1), (1, 0), (0, -1)]]
    }

# 사각지대 count 함수
def check_blind(office):
    return sum(row.count(0) for row in office)

# CCTV 감시 영역 표시 함수
def trans_office(office, pos, directions):
    x, y = pos
    for dx, dy in directions:
        nx, ny = x + dx, y + dy
        while 0 <= nx < len(office) and 0 <= ny < len(office[0]):
            if office[nx][ny] == 6:
                break
            if office[nx][ny] == 0:
                office[nx][ny] = '#'
            nx += dx
            ny += dy
    return office

# DFS 백트래킹
def dfs(idx, office_state, cctvs, cctv_dirs, min_blind):
    if idx == len(cctvs):
        return min(min_blind, check_blind(office_state))

    x, y, cctv = cctvs[idx]
    for directions in cctv_dirs[cctv]:
        copied = copy.deepcopy(office_state)
        trans_office(copied, (x, y), directions)
        min_blind = dfs(idx + 1, copied, cctvs, cctv_dirs, min_blind)

    return min_blind

# 메인 로직 함수
def solve():
    N, M, office, cctvs = parse_input()
    cctv_dirs = get_cctv_directions()
    result = dfs(0, office, cctvs, cctv_dirs, float('inf'))
    print(result)

# 실행
solve()


# CCTV 문제 풀이 방식 비교

## 그리디 (Greedy) 방식

### 개념
- 매 순간 가장 최적인 선택을 하여 전체 문제를 해결하려는 전략
- 각 CCTV에 대해 가능한 방향 중 사각지대가 가장 적은 방향 하나만 선택
- 이 선택을 CCTV 순서대로 누적 적용

### 특징
- 로컬 최적(Local Optimum)을 선택해 글로벌 최적(Global Optimum)을 기대
- 한 번 선택한 방향은 되돌리지 않음
- 이전 선택이 다음 CCTV의 선택에 영향을 미침

### 장점
- 구현이 비교적 간단함
- 연산 속도가 빠름 (탐색 깊이 없음)
- 입력 크기가 큰 경우에도 현실적인 실행 속도 확보 가능

### 단점
- 전체 조합을 고려하지 않기 때문에, 최적의 해를 보장하지 않음
- CCTV 간 감시 구역이 겹칠 경우, 감시 낭비 발생 가능
- 특정 상황에서는 정답과 큰 오차가 날 수도 있음

### 사용하기 좋은 경우
- 정확한 정답보다 빠른 근사값이 중요한 경우
- CCTV 수가 많고 시간 복잡도가 제한되는 문제
- 실시간 또는 제한된 환경에서의 최적화 문제

### 추가 정리
- CCTV 선택이 다른 것에 영향이 없다면 사용하기 좋은 알고리즘
- 하지만, 다른 CCTV에 영향을 주는 문제에서는 전체 최적을 보장하지 않음

---

## DFS (백트래킹) 방식

### 개념
- 모든 CCTV의 방향 조합을 완전 탐색하여 사각지대를 최소화하는 방향 조합을 찾는 방식
- 각 CCTV가 가능한 모든 방향을 재귀적으로 시도
- 방향을 선택한 후 결과를 저장하고 다음 CCTV로 진행
- 이후 다시 되돌려서 다른 방향을 시도 (백트래킹)

### 특징
- 전역 최적 해(Global Optimum)를 탐색
- CCTV 간 영향, 감시 중복 등을 모두 고려한 방향 조합을 탐색
- 탐색 트리는 깊이 = CCTV 개수, 가지 = 방향 수

### 장점
- 최적의 정답 보장
- 모든 경우를 시도하므로 감시 낭비 없음
- CCTV 간 영향을 완벽하게 고려 가능

### 단점
- 시간 복잡도가 매우 높음 (최악의 경우 O(4ⁿ), n = CCTV 수)
- CCTV 수가 많을 경우 실행 시간이 매우 길어짐
- 구현 복잡도가 상대적으로 높음

### 사용하기 좋은 경우
- 정확한 정답이 필요한 문제
- CCTV 수가 적당히 적은 경우 (20 이하 추천)
- 탐색 시간이 중요하지 않은 상황


# 문제 풀이

## 백트랙킹 문제를 판단하는 방법

### 조건
1. 모든 경우의 수
2. 완전 탐색이지만 조건이 명확
3. 조건이 적음

### DFS
- 깊게 들어가서 ```끝까지 도달``` 후 ```되돌아오면서``` 다른 경로를 탐색하는 방식

### 백트랙킹
- DFS 도중 '더 이상 해가 아님'을 알게 되면 되돌아가는 전략

### 코드 구조

In [ ]:
# 기본
def dfs(선택한_수):
    if 종료조건:
        정답 체크
        return

    for 가능한_선택 in 선택지:
        상태 저장
        dfs(다음 선택)
        상태 복구  # 백트래킹

# 예시
def dfs(path, depth):
    if depth == 2:
        print(path)
        return

    for i in range(1, 4):
        if i in path:  # 중복 방지
            continue
        dfs(path + [i], depth + 1)

def dfs(idx, office_state, cctvs, cctv_dirs, min_blind):
    if idx == len(cctvs): # 종료조건 cctv를 모두 건들였을 때
        return min(min_blind, check_blind(office_state)) # 반환 값 최소 blind 값, 그 상태의 office 상태

    x, y, cctv = cctvs[idx]
    for directions in cctv_dirs[cctv]: #각 cctv의 방향 선택지
        copied = copy.deepcopy(office_state)# 배열을 복사하여 상태 복귀의 역할을 해줌 -> 최적이 아니어도 원본 값에 영향이 없음
        trans_office(copied, (x, y), directions)# 방향만 바뀜
        min_blind = dfs(idx + 1, copied, cctvs, cctv_dirs, min_blind)# 방향이 바뀜 office 상태를 보내면서 다음의 선택을 함

    return min_blind